In [ ]:
from IPython.display import clear_output
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import re
import dropbox
import sys
import pandas as pd
import numpy as np
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
import logging
import datetime
import glob
import json
import scipy
import matplotlib as mpl

from subprocess import call
from tifffile import imwrite
from tqdm import tqdm
from pathlib import Path
from amftrack.util.dbx import (
    upload_folder,
    download,
    read_saved_dropbox_state,
    save_dropbox_state,
    load_dbx,
    get_dropbox_folders_prince,
    get_dropbox_video_folders,
    download_video_folders_drop,
    download_analysis_folders_drop,
)
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer
from amftrack.pipeline.launching.run_super import run_parallel_flows
from amftrack.pipeline.functions.transport_processing.high_mag_videos.plot_data import (
    plot_summary,
    save_raw_data,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.high_mag_analysis import (
    HighmagDataset,
    VideoDataset,
    EdgeDataset,
    index_videos_dropbox_new,
    analysis_run,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.kymo_class import (
    KymoVideoAnalysis,
    KymoEdgeAnalysis,
)
from amftrack.util.dbx import (upload
)
%load_ext autoreload
%autoreload 2
%matplotlib widget

from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
    load_graphs,
    Edge,
    Node,

)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    reconstruct_image_from_general,
    plot_full,
    plot_edge_color_value,
)
from amftrack.transport.align_video_network import identify_nodes
logging.basicConfig(stream=sys.stdout, level=logging.debug)
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
mpl.rcParams["figure.dpi"] = 100
from amftrack.pipeline.functions.transport_processing.high_mag_videos.register_videos import *


In [ ]:
plate_id = "310_20230830"
plate_id_video = "20230903_Plate310"
videos_folder = "/projects/0/einf914/videos/"


In [ ]:
indexes = {
"20230901_Plate310" : 20,
"20230902_Plate310" : 32,
"20230904_Plate310" : 44,
    
}

In [ ]:
analysis_folder = "/projects/0/einf914/analysis_videos/CocoTransport/"
analysis_folder = f"{analysis_folder}{plate_id_video}/"

img_infos = glob.glob(f"{analysis_folder}/**/video_data.json", recursive=True)
vid_anls_frame = pd.DataFrame()
add_infos = []
for address in img_infos:
    add_infos.append(pd.read_json(address, orient="index").T)
vid_anls_frame = pd.concat([vid_anls_frame] + add_infos, ignore_index=True)

vid_anls_frame = vid_anls_frame.sort_values("unique_id").reset_index(drop=True)
vid_anls_frame_select = vid_anls_frame.loc[vid_anls_frame["plate_id"] == plate_id_video]

In [ ]:
analysis_folder = "/projects/0/einf914/analysis_videos/CocoTransport/"
analysis_folder = f"{analysis_folder}{plate_id_video}/"

img_infos = glob.glob(f"{analysis_folder}/**/video_data_network.json", recursive=True)
vid_anls_frame = pd.DataFrame()
add_infos = []
for address in img_infos:
    add_infos.append(pd.read_json(address, orient="index").T)
vid_anls_frame = pd.concat([vid_anls_frame] + add_infos, ignore_index=True)

vid_anls_frame = vid_anls_frame.sort_values("unique_id").reset_index(drop=True)
vid_anls_frame_select_network = vid_anls_frame.loc[vid_anls_frame["plate_id"] == plate_id_video]

In [ ]:
vid_anls_frame_merged = vid_anls_frame_select.merge(vid_anls_frame_select_network[['xpos_network','ypos_network','unique_id']],on = 'unique_id')

In [ ]:
analysis_folder_root = "/projects/0/einf914/analysis_videos/"


In [ ]:
vid_anls_frame_merged["analysis_folder"] = analysis_folder_root
vid_anls_frame_merged["videos_folder"] = [
    str(Path(videos_folder) / entry["folder"])
    for index, entry in vid_anls_frame_merged.iterrows()
]

In [ ]:
data_obj = HighmagDataset(vid_anls_frame_merged, analysis_folder_root, videos_folder)

In [ ]:
# directory_targ = os.path.join(directory_scratch, "stitch_temp2") + "/"
directory_targ = '/projects/0/einf914/transport/'
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

In [ ]:
folders = all_folders.loc[all_folders["unique_id"] == plate_id]
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]

In [ ]:
folders = folders.sort_values(by="datetime")

exp = Experiment(directory_targ)
i = 44
exp.load(folders.iloc[i : i + 2], suffix="_width")
for t in range(exp.ts):
    exp.load_tile_information(t)

In [ ]:
mpl.rcParams["figure.dpi"] = 50

from random import choice
vid_obj = data_obj.video_objs[14]
vid_obj.plot_speed_arrows(plot_both=True, video_txt_size=30)


In [ ]:
first_frame = vid_obj.get_first_frame()
weight = 0.1
fig,ax = plt.subplots()
ax.imshow(first_frame)
for edge in vid_obj.edge_objs:
    begin = edge.edge_name.split(',')[0][1:]
    end = edge.edge_name.split(',')[1][1:-1]
    ax.text(0.9*edge.edge_infos['edge_ypos_1']+0.1*edge.edge_infos['edge_ypos_2'],
            0.9*edge.edge_infos['edge_xpos_1']+0.1*edge.edge_infos['edge_xpos_2'],
        str(begin),
        color="white",
    )
    ax.text(0.9*edge.edge_infos['edge_ypos_2']+0.1*edge.edge_infos['edge_ypos_1'],
            0.9*edge.edge_infos['edge_xpos_2']+0.1*edge.edge_infos['edge_xpos_1'],
        str(end),
        color="white",
    )

In [ ]:
from PIL import Image, ImageDraw, ImageFont
draw = ImageDraw.Draw(first_frame_img)

# Optionally, specify a font for the text
# font = ImageFont.truetype("path_to_font.ttf", font_size)

for edge in vid_obj.edge_objs:
    begin = edge.edge_name.split(',')[0][1:]
    end = edge.edge_name.split(',')[1][1:-1]
    ypos_1 = edge.edge_infos['edge_ypos_1']
    xpos_1 = edge.edge_infos['edge_xpos_1']
    ypos_2 = edge.edge_infos['edge_ypos_2']
    xpos_2 = edge.edge_infos['edge_xpos_2']

    # Calculate positions for the start and end texts
    start_pos = (0.9*ypos_1+0.1*ypos_2, 0.9*xpos_1+0.1*xpos_2)
    end_pos = (0.9*ypos_2+0.1*ypos_1, 0.9*xpos_2+0.1*xpos_1)

    # Draw the start and end texts on the image
    draw.text(start_pos, str(begin), fill="white")  # , font=font) if using a custom font
    draw.text(end_pos, str(end), fill="white")  # , font=font) if using a custom font

# Save the modified image or display it
first_frame_img.show()

In [ ]:
edge = vid_obj.edge_objs[0]

In [ ]:
edge_begin = edge.mean_data["network_begin"]
edge_end = edge.mean_data["network_end"]

In [ ]:
network_edge = Edge(edge_begin,edge_end,exp)

In [ ]:
network_edge

In [ ]:
positions = np.array(vid_obj.dataset[["xpos_network", "ypos_network"]])
positions_list = [positions.tolist()]
window = np.array([100,100])
begin= (positions-window).astype(int)
end = (positions+window).astype(int)
region=[[begin[0],begin[1]],[end[0],end[1]]]
# region = [[100, 100], [2000,2000]]

In [ ]:
shiftx = vid_obj.img_dim[0]*vid_obj.space_res/1.725/2
shifty = vid_obj.img_dim[1]*vid_obj.space_res/1.725/2

In [ ]:
segments = []
for i in range(len(vid_obj.edge_objs)):
    edge = vid_obj.edge_objs[i]
    edge_dat_adr = (
        Path(f"{analysis_folder_root}{edge.mean_data['folder']}")
        / f"edges_data.csv"
    )
    edges = pd.read_csv(edge_dat_adr)
    edge.edge_infos = edges[edges['edge_name']==edge.edge_name].iloc[0]
    edge.edge_infos['edge_xpos_1']   
    x_pos_video = edge.mean_data['xpos_network']
    y_pos_video = edge.mean_data['ypos_network']

    x_pos1 = edge.edge_infos['edge_xpos_1']*edge.space_res/1.725+x_pos_video-shiftx
    x_pos2 = edge.edge_infos['edge_xpos_2']*edge.space_res/1.725+x_pos_video-shiftx
    y_pos1 = edge.edge_infos['edge_ypos_1']*edge.space_res/1.725+y_pos_video-shifty
    y_pos2 = edge.edge_infos['edge_ypos_2']*edge.space_res/1.725+y_pos_video-shifty
    dist = np.linalg.norm(np.array([x_pos1,y_pos1])-np.array([x_pos2,y_pos2]))
    if dist>40:
        segments.append([[x_pos1,y_pos1],[x_pos2,y_pos2]])
        # print(dist)

In [ ]:
edges = get_all_edges(exp, t)

edges = [edge for edge in edges if dist_edge(edge,positions,t)<=100]
pixels = [pixel for edge in edges for pixel in edge.pixel_list(t)]
pixels = [pixel for pixel in pixels if np.linalg.norm(pixel-positions)<=100]    

In [ ]:
segment_points = []
for begin, end in segments:
    # Include the start point, interpolated points, and the end point
    interpolated_points = interpolate_points(begin, end)
    segment_points.extend(interpolated_points)
    segment_points.append(end)  # Ensure the end point is included

segment_points = np.array(segment_points)


In [ ]:
positions_list = [positions.tolist()]
plt.close("all")
t = 0
vmax = 9
vmin = 3
nodes = get_all_nodes(exp, t)
edges = get_all_edges(exp, t)
edges = [edge for edge in edges if dist_edge(edge,positions,t)<=150]
downsizing = 1

plot_full_video(
    exp,
    t,
    downsizing=downsizing,
    points=positions_list,
    video_num=[0],
    edges=edges,
    dilation=5,
    region = region,
    segments = segments
)

In [ ]:
fig,ax = plt.subplots()

# Plotting point cloud
px, py = zip(*pixels)  # Unpacking points into x and y coordinates
ax.scatter(px, py, color='blue')  # Plot points in blue
px, py = zip(*segment_points)  # Unpacking points into x and y coordinates
ax.scatter(px, py, color='red')  # Plot points in blue

In [ ]:
Rfound,tfound = register_rot_trans(vid_obj,exp,t,dist= 100)

In [ ]:
transformed_points = np.transpose(Rfound @ np.transpose(segment_points)[[0,1],:] + tfound[:, np.newaxis])


In [ ]:
fig,ax = plt.subplots()

# Plotting point cloud
px, py = zip(*pixels)  # Unpacking points into x and y coordinates
ax.scatter(px, py, color='blue')  # Plot points in 
px, py = zip(*transformed_points)  # Unpacking points into x and y coordinates
ax.scatter(px, py, color='red')  # Plot points in blue

In [ ]:
edges = get_all_edges(exp, t)

edges = [edge for edge in edges if dist_edge(edge,transform(positions,Rfound,tfound),t)<=100]
network_edge_segments = [edge.pixel_list(t) for edge in edges]
network_edge_segments = [[pixel for pixel in pixels if np.linalg.norm(pixel-positions)<=1.5*dist] for pixels in network_edge_segments]
network_edge_segments = [[pixels[0],pixels[-1]] for pixels in network_edge_segments]

In [ ]:
def initialize_transformation():
    return np.array([[1,0],[0,1]]), np.array([0,0])

In [ ]:
Rcurrent, tcurrent = initialize_transformation()
mapping, dist, Rfound, tfound = make_whole_mapping(vid_obj, exp, t, dist=100, R=Rcurrent, trans=tcurrent)


In [ ]:
fig,ax = plt.subplots()

for segment in network_edge_segments:
    x_values = [segment[0][0], segment[1][0]]  # Extracting x-coordinates of the segment's endpoints
    y_values = [segment[0][1], segment[1][1]]  # Extracting y-coordinates of the segment's endpoints
    ax.plot(x_values, y_values, marker='o')  # Plotting the segment with points at the endpoints

px, py = zip(*transformed_points)  # Unpacking points into x and y coordinates
ax.scatter(px, py, color='red')  # Plot points in blue